# GDELT GKG TO PRIO GRID TO INDIGENOUS

1.   Here we are trying to find out the PRIO GRID GIDs for the locations that appear in the news articles and then map those PRIO GRID IDs to whether those GRID IDs are indegenous locations or not.
2.   We are basically taking the latitudes and longitudes of all the locations in the news articles and mapping them to PRIO GRID latitudes and longitudes to get the corresponding Grid IDs of all the locations. At the end, we will have the Grid IDs to which the locations appearing in the news articles belong to. Then we will map these GRID IDs to indegenous data to find which of the found out PRIO GRID IDs are indegenous.
3.   We will be considering only location types 3 and 4 - which are US cities and World cities that appear in these news articles.

#### First what we will do is get the data from the GDELT GKG for the countries of our choice. For this we should first write an Athena code in Athena.
#### The Athena code can be run directly from SageMaker by using 'PyAthena'
#### Here we are pulling the data for some countries in Central America (FIPS codes 'GT','CS','HO','PM','NU','BH','ES') for year 2021 from the GDELT GKG locations table and storing that data in a dataframe called 'df'. You can change the countries list for which you want the data. Then run the Athena code in AWS Athena through PyAthena.
#### Now our data is present in 'df'. 

#### So first lets import and all the other libraries that we want.


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Now we need to pull the data using PyAthena. You need to change the names of the countries that you want data for in the below code.

In [18]:
import sys
!{sys.executable} -m pip install PyAthena

In [26]:
from pyathena import connect
import pandas as pd
conn = connect(s3_staging_dir='s3://kcsra/Athena Test/',
               region_name='us-east-1')


# df = pd.read_sql("""SELECT * 
#                     FROM "gdeltgkg"."locations"
#                     WHERE year = '2021' and month(date) = 1""", conn)

# df = pd.read_sql("""SELECT * 
#                     FROM "gdeltgkg"."locations"
#                     WHERE country in ('BH','CS','ES','GT','HO','NU','PM') and year = '2021'""", conn)

df = pd.read_sql("""SELECT * 
                    FROM "gdeltgkg"."locations"
                    WHERE country = 'NG' AND year = '2021'""", conn)
df

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,tone,dates,amounts,translationinfo,country,year
0,20210923051500-765,2021-09-23 05:15:00,1,iaea.org,https://www.iaea.org/newscenter/news/niger-sig...,None,"[{location_type=1, location_fullname=Niger, lo...",[{organization=Niger Country Programme Framewo...,"[{theme=TAX_FNCACT_DEPUTY, character_offset=11...","[{person=Mahamane Sani Mahamadou, character_of...","{tone=3.1496062992126, positive_score=3.149606...","[{date_resolution=1, month=0, day=0, year=1969...","[{amount=5, object=priority areas, character_o...",None,NG,2021
1,20210919184500-673,2021-09-19 18:45:00,1,arynews.tv,https://arynews.tv/biden-talks-macron-amid-sub...,None,"[{location_type=1, location_fullname=Britain, ...","[{organization=French European Affairs, charac...","[{theme=TAX_ETHNICITY_MEND, character_offset=1...","[{person=Rob Bauer, character_offset=3587}, {p...","{tone=-1.98757763975155, positive_score=2.9813...",None,"[{amount=2, object=television, character_offse...",None,NG,2021
2,20210921054500-180,2021-09-21 05:45:00,1,allafrica.com,https://allafrica.com/stories/202109210016.html,None,"[{location_type=1, location_fullname=Nigerian,...",None,[{theme=WB_566_ENVIRONMENT_AND_NATURAL_RESOURC...,"[{person=Kadir Hassan, character_offset=1673},...","{tone=-4.51612903225806, positive_score=1.9354...",None,"[{amount=850, object=former Boko Haram fighter...",None,NG,2021
3,20210924040000-551,2021-09-24 04:00:00,1,naija247news.com,https://www.naija247news.com/2021/09/24/enugu-...,None,"[{location_type=4, location_fullname=Jigawa, K...","[{organization=National Offices Of The Party, ...","[{theme=TAX_FNCACT_DEPUTY, character_offset=21...","[{person=Boni Haruna, character_offset=2521}, ...","{tone=1.33928571428571, positive_score=3.125, ...",None,None,None,NG,2021
4,20210927074500-379,2021-09-27 07:45:00,1,albawaba.com,https://www.albawaba.com/opinion/climate-chang...,"[{count_type=AFFECT, count=3000000, object_typ...","[{location_type=4, location_fullname=Nile Rive...",[{organization=Masters In International Affair...,"[{theme=RURAL, character_offset=4749}, {theme=...","[{person=Kristalina Georgieva, character_offse...","{tone=-3.81231671554252, positive_score=3.2258...",None,"[{amount=40000000, object=people were uprooted...",None,NG,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53525,20210722033000-161,2021-07-22 03:30:00,1,msn.com,https://www.msn.com/en-xl/africa/other/kenya-t...,None,"[{location_type=1, location_fullname=Jordan, l...",[{organization=International Development Assoc...,"[{theme=HEALTH_VACCINATION, character_offset=1...","[{person=Axel Van Trotsenburg, character_offse...","{tone=2.96896086369771, positive_score=4.85829...",None,"[{amount=51, object=developing countries, char...",None,NG,2021
53526,20210516201500-624,2021-05-16 20:15:00,1,tap.info.tn,https://www.tap.info.tn/en/Portal-Politics/140...,"[{count_type=WOUND, count=23, object_type=wome...","[{location_type=4, location_fullname=Tunis, (T...","[{organization=Foreign Affairs Ministry, chara...","[{theme=CRISISLEX_CRISISLEXREC, character_offs...","[{person=Othman Jerandi, character_offset=233}...","{tone=-0.48780487804878, positive_score=1.9512...",None,"[{amount=145, object=on Sunday, character_offs...",None,NG,2021
53527,20210125033000-104,2021-01-25 03:30:00,1,thetidenewsonline.com,http://www.thetidenewsonline.com/2021/01/25/co...,None,"[{location_type=4, location_fullname=Kogi, Nig...","[{organization=National Electoral Commission, ...","[{theme=KILL, character_offset=1223}, {theme=B...","[{person=Joe Biden, character_offset=78}]","{tone=-5.19480519480519, positive_score=4.2207...",None,None,None,NG,2021
53528,20210125033000-309,2021-01-25 03:30:00,1,today.rtl.lu,https://today.rtl.lu/news/world/a/1660415.html,"[{count_type=KILL, count=6, object_type=soldie...","[{location_type=1, location_fullname=Mali, loc...","[{organization=Group 

#### Now lets look at the shape and head of our data

In [27]:
df.shape

(53530, 16)

In [28]:
df.head(2)

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,tone,dates,amounts,translationinfo,country,year
0,20210923051500-765,2021-09-23 05:15:00,1,iaea.org,https://www.iaea.org/newscenter/news/niger-sig...,None,"[{location_type=1, location_fullname=Niger, lo...",[{organization=Niger Country Programme Framewo...,"[{theme=TAX_FNCACT_DEPUTY, character_offset=11...","[{person=Mahamane Sani Mahamadou, character_of...","{tone=3.1496062992126, positive_score=3.149606...","[{date_resolution=1, month=0, day=0, year=1969...","[{amount=5, object=priority areas, character_o...",None,NG,2021
1,20210919184500-673,2021-09-19 18:45:00,1,arynews.tv,https://arynews.tv/biden-talks-macron-amid-sub...,None,"[{location_type=1, location_fullname=Britain, ...","[{organization=French European Affairs, charac...","[{theme=TAX_ETHNICITY_MEND, character_offset=1...","[{person=Rob Bauer, character_offset=3587}, {p...","{tone=-1.98757763975155, positive_score=2.9813...",None,"[{amount=2, object=television, character_offse...",None,NG,2021


#### Here we are cleaning the 'locations' column in the dataset. We are removing '[' and ']' from each row of the 'location' column

In [7]:
df['locations'] = df['locations'].str.replace("[", "")
df['locations'] = df['locations'].str.replace("]", "")
df.head(1)

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,tone,dates,amounts,translationinfo,country,year
0,20211020203000-991,2021-10-20 20:30:00,1,southgatearc.org,http://www.southgatearc.org/bands/10metres/202...,None,"{location_type=1, location_fullname=French Gui...","[{organization=Italian Dxpedition Team, charac...","[{theme=TAX_ETHNICITY_FRENCH, character_offset...","[{person=Tony Dixon, character_offset=10004}]","{tone=-0.686641697877653, positive_score=0.249...","[{date_resolution=4, month=10, day=21, year=0,...","[{amount=10, object=Metres, character_offset=2...",None,NG,2021


#### This is how each entry in the 'locations' column looks like, a bunch of text with location information that needs to be cleaned

In [29]:
df['locations'][0]

'[{location_type=1, location_fullname=Niger, location_countrycode=NG, location_adm1code=NG, location_adm2code=, location_latitude=16, location_longitude=8, location_featureid=NG, character_offset=90}, {location_type=1, location_fullname=Niger, location_countrycode=NG, location_adm1code=NG, location_adm2code=, location_latitude=16, location_longitude=8, location_featureid=NG, character_offset=198}, {location_type=1, location_fullname=Niger, location_countrycode=NG, location_adm1code=NG, location_adm2code=, location_latitude=16, location_longitude=8, location_featureid=NG, character_offset=551}]'

#### Let us first split the 'locations' column at the pattern '}, {' to get different locations in each news article

In [51]:
s = df['locations'].str.split(pat = "}, {")
s

0          [{location_type=1, location_fullname=Chile, lo...
1          [{location_type=4, location_fullname=Santiago,...
2          [{location_type=1, location_fullname=Chile, lo...
3          [{location_type=1, location_fullname=Bahamas, ...
4          [{location_type=1, location_fullname=Chile, lo...
                                 ...                        
1024744    [{location_type=1, location_fullname=Russia, l...
1024745    [{location_type=4, location_fullname=Whitebrid...
1024746    [{location_type=1, location_fullname=Chile, lo...
1024747    [{location_type=1, location_fullname=Chile, lo...
1024748    [{location_type=3, location_fullname=Grosse Po...
Name: locations, Length: 1024749, dtype: object

#### Let us define a function called 'func' that will give only location types 3 and 4 for each news article

In [52]:
def func(x):
  lst = []
  if isinstance(x, list):
    for i in range(0, len(x)):
      if ("location_type=3" in x[i]) or ("location_type=4" in x[i]):
        lst.append(x[i])
      else:
        continue
  return lst

#### Now let us apply the above defined function to the 'locations' column to get only location types 3 and 4

In [53]:
s_only_3_and_4 = s.apply(lambda x: func(x))
s_only_3_and_4

0                                                         []
1          [{location_type=4, location_fullname=Santiago,...
2                                                         []
3          [location_type=4, location_fullname=Kabul, Kab...
4          [location_type=4, location_fullname=Santiago, ...
                                 ...                        
1024744    [location_type=4, location_fullname=Providenci...
1024745    [{location_type=4, location_fullname=Whitebrid...
1024746                                                   []
1024747    [location_type=4, location_fullname=Edinburgh,...
1024748    [{location_type=3, location_fullname=Grosse Po...
Name: locations, Length: 1024749, dtype: object

#### Now s_only_3_and_4 contains only the locations of location type 3 and 4

#### Let us define a function 'lat_long_tuples' that extracts the latitudes and longitude values from each location

In [54]:
def lat_long_tuples(x):
  lst = []
  for i in range(0, len(x)):
    lat_index = x[i].find("location_latitude")
    long_index = x[i].find("location_longitude")
    lat_long = (float(x[i][lat_index + 18:].split(",")[0]), float(x[i][long_index + 19:].split(",")[0]))
    lst.append(lat_long)
  return lst

#### Now let us apply this function on s_only_3_and_4 to get latitudes and longitudes from each location

In [55]:
lat_long_tuples_list = s_only_3_and_4.apply(lambda x: lat_long_tuples(x))
lat_long_tuples_list

0                                                         []
1                                       [(-33.45, -70.6667)]
2                                                         []
3                                       [(34.5167, 69.1833)]
4                                       [(-33.45, -70.6667)]
                                 ...                        
1024744    [(25.0936, -100.625), (25.0936, -100.625), (-2...
1024745    [(56.65, -4.0), (56.65, -4.0), (57.4167, -3.38...
1024746                                                   []
1024747    [(55.95, -3.2), (33.7, 73.1667), (33.7, 73.166...
1024748    [(42.3759, -82.9374), (41.9, 12.4833), (25.252...
Name: locations, Length: 1024749, dtype: object

#### Now we have the latitudes and longitudes for each location in each news article, but we need to convert these values to values in 'PRIO_GRID' spline file to get the 'GIDs' of the respective grids. So let us write a function to convert these lat long values to the nearest grid lat long values - let us call this function 'lat_long_to_PG_lat_long' - out lat long values to PRIO GRID lat long values.

In [56]:
def lat_long_to_PG_lat_long(x):
  lst = []
  for i in range(0, len(x)):
    v = []
    for j in range(0, 2):

      if x[i][j] >= 0:

        if x[i][j] % 1 >= 0 and x[i][j] % 1 <= 0.5:
          v.append(int(x[i][j]) + 0.25)
        else:
          v.append(int(x[i][j]) + 0.75)


      if x[i][j] < 0:
        if x[i][j] % 1 >= 0 and x[i][j] % 1 <= 0.5:
          v.append(int(x[i][j]) - 0.75)
        else:
          v.append(int(x[i][j]) - 0.25)
        
    lst.append(v)
  
  return lst

#### Now let us apply the function to out lat long to get the PRIO GRID compatible lat longs

In [57]:
final_lat_long = lat_long_tuples_list.apply(lambda x: lat_long_to_PG_lat_long(x))
final_lat_long

0                                                         []
1                                         [[-33.25, -70.75]]
2                                                         []
3                                           [[34.75, 69.25]]
4                                         [[-33.25, -70.75]]
                                 ...                        
1024744    [[25.25, -100.75], [25.25, -100.75], [-2.25, -...
1024745    [[56.75, -4.75], [56.75, -4.75], [57.25, -3.25...
1024746                                                   []
1024747    [[55.75, -3.25], [33.75, 73.25], [33.75, 73.25...
1024748    [[42.25, -82.75], [41.75, 12.25], [25.25, 55.2...
Name: locations, Length: 1024749, dtype: object

## Connecting these latitudes and longitudes to PRIO GRID to get the corresponding Grid IDs

#### Let us first read the PRIO GRID Spine file that contains all the lat longs and corresponding GIDs

In [58]:
# Here change within brackets to your PRIO GRID file name or path

PG_Spine = pd.read_csv("PRIO GRID spine.csv")

In [59]:
PG_Spine.head()

,gid,lon,lat
0,49182,-69.25,-55.75
1,49183,-68.75,-55.75
2,49184,-68.25,-55.75
3,49185,-67.75,-55.75
4,49186,-67.25,-55.75


### Now let us concatenate the df and final_lat_long into one dataframe

In [60]:
GDELT_TO_PRIOGRID = pd.concat([df, final_lat_long], axis=1)
GDELT_TO_PRIOGRID.head(20)

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,tone,dates,amounts,translationinfo,country,year,locations
0,20211202213000-16,2021-12-02 21:30:00,1,wmal.com,https://www.wmal.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,[]
1,20211202213000-102,2021-12-02 21:30:00,1,redstate.com,https://redstate.com/nick-arama/2021/12/02/san...,None,"{location_type=4, location_fullname=Santiago, ...","[{organization=Sheriff Department, character_o...","[{theme=KILL, character_offset=142}, {theme=TA...","[{person=Hannah Reed, character_offset=1871}, ...","{tone=-3.85395537525355, positive_score=0.6085...",None,None,None,CI,2021,"[[-33.25, -70.75]]"
2,20211202213000-508,2021-12-02 21:30:00,1,kfru.com,https://www.kfru.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,[]
3,20211202213000-769,2021-12-02 21:30:00,1,thediplomat.com,https://thediplomat.com/2021/12/afghanistans-t...,"[{count_type=ARREST, count=2014, object_type=,...","{location_type=1, location_fullname=Bahamas, l...","[{organization=United Nations, character_offse...","[{theme=TAX_FNCACT_CANDIDATES, character_offse...","[{person=Ghulam Isaczai, character_offset=1780...","{tone=-1.9047619047619, positive_score=1.48148...","[{date_resolution=1, month=0, day=0, year=1949...","[{amount=2, object=situations, character_offse...",None,CI,2021,"[[34.75, 69.25]]"
4,20211202213000-843,2021-12-02 21:30:00,1,citywireamericas.com,https://citywireamericas.com/news/climate-chan...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=National Centers, character_off...","[{theme=NATURAL_DISASTER_EXTREME_WEATHER, char...","[{person=Michelle Dunstan, character_offset=21...","{tone=-1.57273918741809, positive_score=2.4246...",None,"[{amount=30, object=separate billion, characte...",None,CI,2021,"[[-33.25, -70.75]]"
5,20211202213000-1181,2021-12-02 21:30:00,1,realwire.com,https://www.realwire.com/releases/New-study-re...,None,"{location_type=1, location_fullname=Rwanda, lo...","[{organization=Portulans Institute, character_...",None,"[{person=Linkedin Youtube, character_offset=65...","{tone=0.931098696461825, positive_score=2.1415...","[{date_resolution=1, month=0, day=0, year=2002...","[{amount=130, object=global economies based on...",None,CI,2021,"[[38.75, -77.25], [38.75, -77.25], [38.75, -77..."
6,20211202213000-1641,2021-12-02 21:30:00,1,theconversation.com,https://theconversation.com/a-new-exoplanet-me...,None,"{location_type=1, location_fullname=Chile, loc...",[{organization=Nasa Transiting Exoplanet Surve...,"[{theme=TAX_ETHNICITY_GERMAN, character_offset...",None,"{tone=0.757575757575757, positive_score=3.0303...",None,"[{amount=1000000, object=of stars being monito...",None,CI,2021,[]
7,20211202213000-1764,2021-12-02 21:30:00,1,wcvb.com,https://www.wcvb.com/article/taco-night-space-...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=Kennedy Space Center, character...","[{theme=TAX_FNCACT_MANAGER, character_offset=5...","[{person=Matthias Maurer, character_offset=606...","{tone=2.44786944696283, positive_score=3.53581...",None,"[{amount=12, object=of the peppers 

### Now let us rename the new column to 'lat_longs;

In [61]:
GDELT_TO_PRIOGRID.columns.values[-1] = "lat_longs"
GDELT_TO_PRIOGRID.head(20)

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,tone,dates,amounts,translationinfo,country,year,lat_longs
0,20211202213000-16,2021-12-02 21:30:00,1,wmal.com,https://www.wmal.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,[]
1,20211202213000-102,2021-12-02 21:30:00,1,redstate.com,https://redstate.com/nick-arama/2021/12/02/san...,None,"{location_type=4, location_fullname=Santiago, ...","[{organization=Sheriff Department, character_o...","[{theme=KILL, character_offset=142}, {theme=TA...","[{person=Hannah Reed, character_offset=1871}, ...","{tone=-3.85395537525355, positive_score=0.6085...",None,None,None,CI,2021,"[[-33.25, -70.75]]"
2,20211202213000-508,2021-12-02 21:30:00,1,kfru.com,https://www.kfru.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,[]
3,20211202213000-769,2021-12-02 21:30:00,1,thediplomat.com,https://thediplomat.com/2021/12/afghanistans-t...,"[{count_type=ARREST, count=2014, object_type=,...","{location_type=1, location_fullname=Bahamas, l...","[{organization=United Nations, character_offse...","[{theme=TAX_FNCACT_CANDIDATES, character_offse...","[{person=Ghulam Isaczai, character_offset=1780...","{tone=-1.9047619047619, positive_score=1.48148...","[{date_resolution=1, month=0, day=0, year=1949...","[{amount=2, object=situations, character_offse...",None,CI,2021,"[[34.75, 69.25]]"
4,20211202213000-843,2021-12-02 21:30:00,1,citywireamericas.com,https://citywireamericas.com/news/climate-chan...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=National Centers, character_off...","[{theme=NATURAL_DISASTER_EXTREME_WEATHER, char...","[{person=Michelle Dunstan, character_offset=21...","{tone=-1.57273918741809, positive_score=2.4246...",None,"[{amount=30, object=separate billion, characte...",None,CI,2021,"[[-33.25, -70.75]]"
5,20211202213000-1181,2021-12-02 21:30:00,1,realwire.com,https://www.realwire.com/releases/New-study-re...,None,"{location_type=1, location_fullname=Rwanda, lo...","[{organization=Portulans Institute, character_...",None,"[{person=Linkedin Youtube, character_offset=65...","{tone=0.931098696461825, positive_score=2.1415...","[{date_resolution=1, month=0, day=0, year=2002...","[{amount=130, object=global economies based on...",None,CI,2021,"[[38.75, -77.25], [38.75, -77.25], [38.75, -77..."
6,20211202213000-1641,2021-12-02 21:30:00,1,theconversation.com,https://theconversation.com/a-new-exoplanet-me...,None,"{location_type=1, location_fullname=Chile, loc...",[{organization=Nasa Transiting Exoplanet Surve...,"[{theme=TAX_ETHNICITY_GERMAN, character_offset...",None,"{tone=0.757575757575757, positive_score=3.0303...",None,"[{amount=1000000, object=of stars being monito...",None,CI,2021,[]
7,20211202213000-1764,2021-12-02 21:30:00,1,wcvb.com,https://www.wcvb.com/article/taco-night-space-...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=Kennedy Space Center, character...","[{theme=TAX_FNCACT_MANAGER, character_offset=5...","[{person=Matthias Maurer, character_offset=606...","{tone=2.44786944696283, positive_score=3.53581...",None,"[{amount=12, object=of the peppers 

### Now let remove the duplicate location lats and longs from each row

In [62]:
def duplicates_remover(x):
    lst = []
    for i in range(0, len(x)):
        if x[i] not in lst:
            lst.append(x[i])
        else:
            continue
    return lst

GDELT_TO_PRIOGRID['lat_longs'] = GDELT_TO_PRIOGRID['lat_longs'].apply(lambda x: duplicates_remover(x))
GDELT_TO_PRIOGRID.head(20)

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,tone,dates,amounts,translationinfo,country,year,lat_longs
0,20211202213000-16,2021-12-02 21:30:00,1,wmal.com,https://www.wmal.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,[]
1,20211202213000-102,2021-12-02 21:30:00,1,redstate.com,https://redstate.com/nick-arama/2021/12/02/san...,None,"{location_type=4, location_fullname=Santiago, ...","[{organization=Sheriff Department, character_o...","[{theme=KILL, character_offset=142}, {theme=TA...","[{person=Hannah Reed, character_offset=1871}, ...","{tone=-3.85395537525355, positive_score=0.6085...",None,None,None,CI,2021,"[[-33.25, -70.75]]"
2,20211202213000-508,2021-12-02 21:30:00,1,kfru.com,https://www.kfru.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,[]
3,20211202213000-769,2021-12-02 21:30:00,1,thediplomat.com,https://thediplomat.com/2021/12/afghanistans-t...,"[{count_type=ARREST, count=2014, object_type=,...","{location_type=1, location_fullname=Bahamas, l...","[{organization=United Nations, character_offse...","[{theme=TAX_FNCACT_CANDIDATES, character_offse...","[{person=Ghulam Isaczai, character_offset=1780...","{tone=-1.9047619047619, positive_score=1.48148...","[{date_resolution=1, month=0, day=0, year=1949...","[{amount=2, object=situations, character_offse...",None,CI,2021,"[[34.75, 69.25]]"
4,20211202213000-843,2021-12-02 21:30:00,1,citywireamericas.com,https://citywireamericas.com/news/climate-chan...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=National Centers, character_off...","[{theme=NATURAL_DISASTER_EXTREME_WEATHER, char...","[{person=Michelle Dunstan, character_offset=21...","{tone=-1.57273918741809, positive_score=2.4246...",None,"[{amount=30, object=separate billion, characte...",None,CI,2021,"[[-33.25, -70.75]]"
5,20211202213000-1181,2021-12-02 21:30:00,1,realwire.com,https://www.realwire.com/releases/New-study-re...,None,"{location_type=1, location_fullname=Rwanda, lo...","[{organization=Portulans Institute, character_...",None,"[{person=Linkedin Youtube, character_offset=65...","{tone=0.931098696461825, positive_score=2.1415...","[{date_resolution=1, month=0, day=0, year=2002...","[{amount=130, object=global economies based on...",None,CI,2021,"[[38.75, -77.25]]"
6,20211202213000-1641,2021-12-02 21:30:00,1,theconversation.com,https://theconversation.com/a-new-exoplanet-me...,None,"{location_type=1, location_fullname=Chile, loc...",[{organization=Nasa Transiting Exoplanet Surve...,"[{theme=TAX_ETHNICITY_GERMAN, character_offset...",None,"{tone=0.757575757575757, positive_score=3.0303...",None,"[{amount=1000000, object=of stars being monito...",None,CI,2021,[]
7,20211202213000-1764,2021-12-02 21:30:00,1,wcvb.com,https://www.wcvb.com/article/taco-night-space-...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=Kennedy Space Center, character...","[{theme=TAX_FNCACT_MANAGER, character_offset=5...","[{person=Matthias Maurer, character_offset=606...","{tone=2.44786944696283, positive_score=3.53581...",None,"[{amount=12, object=of the peppers will, chara...",None,CI,2021,"[[

### Now let us create a new column which shows the number of unique type 3 and 4 locations for each news article

In [63]:
GDELT_TO_PRIOGRID['Number of unique type 3 and 4 locations'] = GDELT_TO_PRIOGRID['lat_longs'].apply(lambda x: len(x))
GDELT_TO_PRIOGRID.head(20)

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,tone,dates,amounts,translationinfo,country,year,lat_longs,Number of unique type 3 and 4 locations
0,20211202213000-16,2021-12-02 21:30:00,1,wmal.com,https://www.wmal.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,[],0
1,20211202213000-102,2021-12-02 21:30:00,1,redstate.com,https://redstate.com/nick-arama/2021/12/02/san...,None,"{location_type=4, location_fullname=Santiago, ...","[{organization=Sheriff Department, character_o...","[{theme=KILL, character_offset=142}, {theme=TA...","[{person=Hannah Reed, character_offset=1871}, ...","{tone=-3.85395537525355, positive_score=0.6085...",None,None,None,CI,2021,"[[-33.25, -70.75]]",1
2,20211202213000-508,2021-12-02 21:30:00,1,kfru.com,https://www.kfru.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,[],0
3,20211202213000-769,2021-12-02 21:30:00,1,thediplomat.com,https://thediplomat.com/2021/12/afghanistans-t...,"[{count_type=ARREST, count=2014, object_type=,...","{location_type=1, location_fullname=Bahamas, l...","[{organization=United Nations, character_offse...","[{theme=TAX_FNCACT_CANDIDATES, character_offse...","[{person=Ghulam Isaczai, character_offset=1780...","{tone=-1.9047619047619, positive_score=1.48148...","[{date_resolution=1, month=0, day=0, year=1949...","[{amount=2, object=situations, character_offse...",None,CI,2021,"[[34.75, 69.25]]",1
4,20211202213000-843,2021-12-02 21:30:00,1,citywireamericas.com,https://citywireamericas.com/news/climate-chan...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=National Centers, character_off...","[{theme=NATURAL_DISASTER_EXTREME_WEATHER, char...","[{person=Michelle Dunstan, character_offset=21...","{tone=-1.57273918741809, positive_score=2.4246...",None,"[{amount=30, object=separate billion, characte...",None,CI,2021,"[[-33.25, -70.75]]",1
5,20211202213000-1181,2021-12-02 21:30:00,1,realwire.com,https://www.realwire.com/releases/New-study-re...,None,"{location_type=1, location_fullname=Rwanda, lo...","[{organization=Portulans Institute, character_...",None,"[{person=Linkedin Youtube, character_offset=65...","{tone=0.931098696461825, positive_score=2.1415...","[{date_resolution=1, month=0, day=0, year=2002...","[{amount=130, object=global economies based on...",None,CI,2021,"[[38.75, -77.25]]",1
6,20211202213000-1641,2021-12-02 21:30:00,1,theconversation.com,https://theconversation.com/a-new-exoplanet-me...,None,"{location_type=1, location_fullname=Chile, loc...",[{organization=Nasa Transiting Exoplanet Surve...,"[{theme=TAX_ETHNICITY_GERMAN, character_offset...",None,"{tone=0.757575757575757, positive_score=3.0303...",None,"[{amount=1000000, object=of stars being monito...",None,CI,2021,[],0
7,20211202213000-1764,2021-12-02 21:30:00,1,wcvb.com,https://www.wcvb.com/article/taco-night-space-...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=Kennedy Space Center, character...","[{theme=TAX_FNCACT_MANAGER, character_offset=5...","[{person=Matthias Maurer, character_offset=606...","{tone=2.44786944696283, positive_score=3.53581...",None,"[{amount=12, 

### Now let us explode the lat_longs columns so that we get one observation per lat_long

In [64]:
cols = GDELT_TO_PRIOGRID.columns
GDELT_TO_PRIOGRID_FINAL = GDELT_TO_PRIOGRID[cols].explode('lat_longs').reset_index(drop = True)
GDELT_TO_PRIOGRID_FINAL.head(20)

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,tone,dates,amounts,translationinfo,country,year,lat_longs,Number of unique type 3 and 4 locations
0,20211202213000-16,2021-12-02 21:30:00,1,wmal.com,https://www.wmal.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,NaN,0
1,20211202213000-102,2021-12-02 21:30:00,1,redstate.com,https://redstate.com/nick-arama/2021/12/02/san...,None,"{location_type=4, location_fullname=Santiago, ...","[{organization=Sheriff Department, character_o...","[{theme=KILL, character_offset=142}, {theme=TA...","[{person=Hannah Reed, character_offset=1871}, ...","{tone=-3.85395537525355, positive_score=0.6085...",None,None,None,CI,2021,"[-33.25, -70.75]",1
2,20211202213000-508,2021-12-02 21:30:00,1,kfru.com,https://www.kfru.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,NaN,0
3,20211202213000-769,2021-12-02 21:30:00,1,thediplomat.com,https://thediplomat.com/2021/12/afghanistans-t...,"[{count_type=ARREST, count=2014, object_type=,...","{location_type=1, location_fullname=Bahamas, l...","[{organization=United Nations, character_offse...","[{theme=TAX_FNCACT_CANDIDATES, character_offse...","[{person=Ghulam Isaczai, character_offset=1780...","{tone=-1.9047619047619, positive_score=1.48148...","[{date_resolution=1, month=0, day=0, year=1949...","[{amount=2, object=situations, character_offse...",None,CI,2021,"[34.75, 69.25]",1
4,20211202213000-843,2021-12-02 21:30:00,1,citywireamericas.com,https://citywireamericas.com/news/climate-chan...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=National Centers, character_off...","[{theme=NATURAL_DISASTER_EXTREME_WEATHER, char...","[{person=Michelle Dunstan, character_offset=21...","{tone=-1.57273918741809, positive_score=2.4246...",None,"[{amount=30, object=separate billion, characte...",None,CI,2021,"[-33.25, -70.75]",1
5,20211202213000-1181,2021-12-02 21:30:00,1,realwire.com,https://www.realwire.com/releases/New-study-re...,None,"{location_type=1, location_fullname=Rwanda, lo...","[{organization=Portulans Institute, character_...",None,"[{person=Linkedin Youtube, character_offset=65...","{tone=0.931098696461825, positive_score=2.1415...","[{date_resolution=1, month=0, day=0, year=2002...","[{amount=130, object=global economies based on...",None,CI,2021,"[38.75, -77.25]",1
6,20211202213000-1641,2021-12-02 21:30:00,1,theconversation.com,https://theconversation.com/a-new-exoplanet-me...,None,"{location_type=1, location_fullname=Chile, loc...",[{organization=Nasa Transiting Exoplanet Surve...,"[{theme=TAX_ETHNICITY_GERMAN, character_offset...",None,"{tone=0.757575757575757, positive_score=3.0303...",None,"[{amount=1000000, object=of stars being monito...",None,CI,2021,NaN,0
7,20211202213000-1764,2021-12-02 21:30:00,1,wcvb.com,https://www.wcvb.com/article/taco-night-space-...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=Kennedy Space Center, character...","[{theme=TAX_FNCACT_MANAGER, character_offset=5...","[{person=Matthias Maurer, character_offset=606...","{tone=2.44786944696283, positive_score=3.53581...",None,"[{amount=12, objec

### Now the lat longs column has some NA values because of explosion. Lets give [0,0] to all NA values

In [65]:
import math
import numpy as np
def remove_nan(x):
    if isinstance(x, list):
        return x
    else:
        return [0,0]

In [66]:
GDELT_TO_PRIOGRID_FINAL['lat_longs'] = GDELT_TO_PRIOGRID_FINAL['lat_longs'].apply(lambda x:remove_nan(x))
GDELT_TO_PRIOGRID_FINAL.head(20)

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,tone,dates,amounts,translationinfo,country,year,lat_longs,Number of unique type 3 and 4 locations
0,20211202213000-16,2021-12-02 21:30:00,1,wmal.com,https://www.wmal.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,"[0, 0]",0
1,20211202213000-102,2021-12-02 21:30:00,1,redstate.com,https://redstate.com/nick-arama/2021/12/02/san...,None,"{location_type=4, location_fullname=Santiago, ...","[{organization=Sheriff Department, character_o...","[{theme=KILL, character_offset=142}, {theme=TA...","[{person=Hannah Reed, character_offset=1871}, ...","{tone=-3.85395537525355, positive_score=0.6085...",None,None,None,CI,2021,"[-33.25, -70.75]",1
2,20211202213000-508,2021-12-02 21:30:00,1,kfru.com,https://www.kfru.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,"[0, 0]",0
3,20211202213000-769,2021-12-02 21:30:00,1,thediplomat.com,https://thediplomat.com/2021/12/afghanistans-t...,"[{count_type=ARREST, count=2014, object_type=,...","{location_type=1, location_fullname=Bahamas, l...","[{organization=United Nations, character_offse...","[{theme=TAX_FNCACT_CANDIDATES, character_offse...","[{person=Ghulam Isaczai, character_offset=1780...","{tone=-1.9047619047619, positive_score=1.48148...","[{date_resolution=1, month=0, day=0, year=1949...","[{amount=2, object=situations, character_offse...",None,CI,2021,"[34.75, 69.25]",1
4,20211202213000-843,2021-12-02 21:30:00,1,citywireamericas.com,https://citywireamericas.com/news/climate-chan...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=National Centers, character_off...","[{theme=NATURAL_DISASTER_EXTREME_WEATHER, char...","[{person=Michelle Dunstan, character_offset=21...","{tone=-1.57273918741809, positive_score=2.4246...",None,"[{amount=30, object=separate billion, characte...",None,CI,2021,"[-33.25, -70.75]",1
5,20211202213000-1181,2021-12-02 21:30:00,1,realwire.com,https://www.realwire.com/releases/New-study-re...,None,"{location_type=1, location_fullname=Rwanda, lo...","[{organization=Portulans Institute, character_...",None,"[{person=Linkedin Youtube, character_offset=65...","{tone=0.931098696461825, positive_score=2.1415...","[{date_resolution=1, month=0, day=0, year=2002...","[{amount=130, object=global economies based on...",None,CI,2021,"[38.75, -77.25]",1
6,20211202213000-1641,2021-12-02 21:30:00,1,theconversation.com,https://theconversation.com/a-new-exoplanet-me...,None,"{location_type=1, location_fullname=Chile, loc...",[{organization=Nasa Transiting Exoplanet Surve...,"[{theme=TAX_ETHNICITY_GERMAN, character_offset...",None,"{tone=0.757575757575757, positive_score=3.0303...",None,"[{amount=1000000, object=of stars being monito...",None,CI,2021,"[0, 0]",0
7,20211202213000-1764,2021-12-02 21:30:00,1,wcvb.com,https://www.wcvb.com/article/taco-night-space-...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=Kennedy Space Center, character...","[{theme=TAX_FNCACT_MANAGER, character_offset=5...","[{person=Matthias Maurer, character_offset=606...","{tone=2.44786944696283, positive_score=3.53581...",None,"[{a

### Now let us split the lat longs 

In [67]:
lat_longs_df = pd.DataFrame(GDELT_TO_PRIOGRID_FINAL['lat_longs'].to_list(), columns = ['lat', 'long'])
lat_longs_df

,lat,long
0,0.00,0.00
1,-33.25,-70.75
2,0.00,0.00
3,34.75,69.25
4,-33.25,-70.75
...,...,...
2832321,25.25,55.25
2832322,41.25,19.75
2832323,42.75,-82.75
2832324,41.75,-87.75


### Now lets join this dataframe with PRIO GRID Spine to get the corresponding GIDs

In [68]:
PG_Spine.head()

,gid,lon,lat
0,49182,-69.25,-55.75
1,49183,-68.75,-55.75
2,49184,-68.25,-55.75
3,49185,-67.75,-55.75
4,49186,-67.25,-55.75


In [69]:
lat_longs_merged = lat_longs_df.merge(PG_Spine, how = "left", left_on = ['lat', 'long'], right_on = ['lat', 'lon'])
GDELT_TO_PRIOGRID_FINAL['GRID IDs'] = lat_longs_merged['gid']
GDELT_TO_PRIOGRID_FINAL.head(20)

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,tone,dates,amounts,translationinfo,country,year,lat_longs,Number of unique type 3 and 4 locations,GRID IDs
0,20211202213000-16,2021-12-02 21:30:00,1,wmal.com,https://www.wmal.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,"[0, 0]",0,NaN
1,20211202213000-102,2021-12-02 21:30:00,1,redstate.com,https://redstate.com/nick-arama/2021/12/02/san...,None,"{location_type=4, location_fullname=Santiago, ...","[{organization=Sheriff Department, character_o...","[{theme=KILL, character_offset=142}, {theme=TA...","[{person=Hannah Reed, character_offset=1871}, ...","{tone=-3.85395537525355, positive_score=0.6085...",None,None,None,CI,2021,"[-33.25, -70.75]",1,81579.0
2,20211202213000-508,2021-12-02 21:30:00,1,kfru.com,https://www.kfru.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,"[0, 0]",0,NaN
3,20211202213000-769,2021-12-02 21:30:00,1,thediplomat.com,https://thediplomat.com/2021/12/afghanistans-t...,"[{count_type=ARREST, count=2014, object_type=,...","{location_type=1, location_fullname=Bahamas, l...","[{organization=United Nations, character_offse...","[{theme=TAX_FNCACT_CANDIDATES, character_offse...","[{person=Ghulam Isaczai, character_offset=1780...","{tone=-1.9047619047619, positive_score=1.48148...","[{date_resolution=1, month=0, day=0, year=1949...","[{amount=2, object=situations, character_offse...",None,CI,2021,"[34.75, 69.25]",1,179779.0
4,20211202213000-843,2021-12-02 21:30:00,1,citywireamericas.com,https://citywireamericas.com/news/climate-chan...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=National Centers, character_off...","[{theme=NATURAL_DISASTER_EXTREME_WEATHER, char...","[{person=Michelle Dunstan, character_offset=21...","{tone=-1.57273918741809, positive_score=2.4246...",None,"[{amount=30, object=separate billion, characte...",None,CI,2021,"[-33.25, -70.75]",1,81579.0
5,20211202213000-1181,2021-12-02 21:30:00,1,realwire.com,https://www.realwire.com/releases/New-study-re...,None,"{location_type=1, location_fullname=Rwanda, lo...","[{organization=Portulans Institute, character_...",None,"[{person=Linkedin Youtube, character_offset=65...","{tone=0.931098696461825, positive_score=2.1415...","[{date_resolution=1, month=0, day=0, year=2002...","[{amount=130, object=global economies based on...",None,CI,2021,"[38.75, -77.25]",1,185246.0
6,20211202213000-1641,2021-12-02 21:30:00,1,theconversation.com,https://theconversation.com/a-new-exoplanet-me...,None,"{location_type=1, location_fullname=Chile, loc...",[{organization=Nasa Transiting Exoplanet Surve...,"[{theme=TAX_ETHNICITY_GERMAN, character_offset...",None,"{tone=0.757575757575757, positive_score=3.0303...",None,"[{amount=1000000, object=of stars being monito...",None,CI,2021,"[0, 0]",0,NaN
7,20211202213000-1764,2021-12-02 21:30:00,1,wcvb.com,https://www.wcvb.com/article/taco-night-space-...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=Kennedy Space Center, character...","[{theme=TAX_FNCACT_MANAGER, character_offset=5...","[{person=Matthias Maurer, character_offset=606...","{tone

### Now lets convert the GRID IDs to integer values

In [70]:
def remove_nan_2(x):
    if str(x) == "nan":
        return 0
    else:
        return int(x)


GDELT_TO_PRIOGRID_FINAL['GRID IDs'] = GDELT_TO_PRIOGRID_FINAL['GRID IDs'].apply(lambda x: remove_nan_2(x))
GDELT_TO_PRIOGRID_FINAL.head()

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,tone,dates,amounts,translationinfo,country,year,lat_longs,Number of unique type 3 and 4 locations,GRID IDs
0,20211202213000-16,2021-12-02 21:30:00,1,wmal.com,https://www.wmal.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,"[0, 0]",0,0
1,20211202213000-102,2021-12-02 21:30:00,1,redstate.com,https://redstate.com/nick-arama/2021/12/02/san...,None,"{location_type=4, location_fullname=Santiago, ...","[{organization=Sheriff Department, character_o...","[{theme=KILL, character_offset=142}, {theme=TA...","[{person=Hannah Reed, character_offset=1871}, ...","{tone=-3.85395537525355, positive_score=0.6085...",None,None,None,CI,2021,"[-33.25, -70.75]",1,81579
2,20211202213000-508,2021-12-02 21:30:00,1,kfru.com,https://www.kfru.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,"[0, 0]",0,0
3,20211202213000-769,2021-12-02 21:30:00,1,thediplomat.com,https://thediplomat.com/2021/12/afghanistans-t...,"[{count_type=ARREST, count=2014, object_type=,...","{location_type=1, location_fullname=Bahamas, l...","[{organization=United Nations, character_offse...","[{theme=TAX_FNCACT_CANDIDATES, character_offse...","[{person=Ghulam Isaczai, character_offset=1780...","{tone=-1.9047619047619, positive_score=1.48148...","[{date_resolution=1, month=0, day=0, year=1949...","[{amount=2, object=situations, character_offse...",None,CI,2021,"[34.75, 69.25]",1,179779
4,20211202213000-843,2021-12-02 21:30:00,1,citywireamericas.com,https://citywireamericas.com/news/climate-chan...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=National Centers, character_off...","[{theme=NATURAL_DISASTER_EXTREME_WEATHER, char...","[{person=Michelle Dunstan, character_offset=21...","{tone=-1.57273918741809, positive_score=2.4246...",None,"[{amount=30, object=separate billion, characte...",None,CI,2021,"[-33.25, -70.75]",1,81579


In [71]:
GDELT_TO_PRIOGRID_FINAL.shape

(2832326, 19)

### Now let us load the indegenous data into a dataframe called INDEG

In [74]:
INDEG = pd.read_csv("indigenous data indig and gid only.csv")
INDEG.head()

,gid,indig
0,49182,0
1,49182,0
2,49182,0
3,49182,0
4,49182,0


### The INDEG Data has many duplicate rows, lets remove them

In [75]:
INDEG = INDEG.drop_duplicates().reset_index(drop = True)
INDEG.head()

,gid,indig
0,49182,0
1,49183,0
2,49184,0
3,49185,0
4,49186,0


### Now let join our GDELT_TO_PRIOGRID_FINAL to INDEG to get indeg column

In [76]:
FINAL_DATASET = GDELT_TO_PRIOGRID_FINAL.merge(INDEG, how = "left", left_on = ['GRID IDs'], right_on = ['gid'])
FINAL_DATASET.head(20)

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,...,dates,amounts,translationinfo,country,year,lat_longs,Number of unique type 3 and 4 locations,GRID IDs,gid,indig
0,20211202213000-16,2021-12-02 21:30:00,1,wmal.com,https://www.wmal.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...",...,"[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,"[0, 0]",0,0,NaN,NaN
1,20211202213000-102,2021-12-02 21:30:00,1,redstate.com,https://redstate.com/nick-arama/2021/12/02/san...,None,"{location_type=4, location_fullname=Santiago, ...","[{organization=Sheriff Department, character_o...","[{theme=KILL, character_offset=142}, {theme=TA...","[{person=Hannah Reed, character_offset=1871}, ...",...,None,None,None,CI,2021,"[-33.25, -70.75]",1,81579,81579.0,0.0
2,20211202213000-508,2021-12-02 21:30:00,1,kfru.com,https://www.kfru.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...",...,"[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,"[0, 0]",0,0,NaN,NaN
3,20211202213000-769,2021-12-02 21:30:00,1,thediplomat.com,https://thediplomat.com/2021/12/afghanistans-t...,"[{count_type=ARREST, count=2014, object_type=,...","{location_type=1, location_fullname=Bahamas, l...","[{organization=United Nations, character_offse...","[{theme=TAX_FNCACT_CANDIDATES, character_offse...","[{person=Ghulam Isaczai, character_offset=1780...",...,"[{date_resolution=1, month=0, day=0, year=1949...","[{amount=2, object=situations, character_offse...",None,CI,2021,"[34.75, 69.25]",1,179779,179779.0,0.0
4,20211202213000-843,2021-12-02 21:30:00,1,citywireamericas.com,https://citywireamericas.com/news/climate-chan...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=National Centers, character_off...","[{theme=NATURAL_DISASTER_EXTREME_WEATHER, char...","[{person=Michelle Dunstan, character_offset=21...",...,None,"[{amount=30, object=separate billion, characte...",None,CI,2021,"[-33.25, -70.75]",1,81579,81579.0,0.0
5,20211202213000-1181,2021-12-02 21:30:00,1,realwire.com,https://www.realwire.com/releases/New-study-re...,None,"{location_type=1, location_fullname=Rwanda, lo...","[{organization=Portulans Institute, character_...",None,"[{person=Linkedin Youtube, character_offset=65...",...,"[{date_resolution=1, month=0, day=0, year=2002...","[{amount=130, object=global economies based on...",None,CI,2021,"[38.75, -77.25]",1,185246,185246.0,0.0
6,20211202213000-1641,2021-12-02 21:30:00,1,theconversation.com,https://theconversation.com/a-new-exoplanet-me...,None,"{location_type=1, location_fullname=Chile, loc...",[{organization=Nasa Transiting Exoplanet Surve...,"[{theme=TAX_ETHNICITY_GERMAN, character_offset...",None,...,None,"[{amount=1000000, object=of stars being monito...",None,CI,2021,"[0, 0]",0,0,NaN,NaN
7,20211202213000-1764,2021-12-02 21:30:00,1,wcvb.com,https://www.wcvb.com/article/taco-night-space-...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=Kennedy Space Center, character...","[{theme=TAX_FNCACT_MANAGER, character_offset=5...","[{person=Matthias Maurer, character_offset=606...",...,None,"[{amount=12, object=of the peppers will, chara...",None,CI,2021,"[32.25, -106.75]",1,175827,175827.0,0.0
8,20211221213000-43,2021-12-21 21:30:00,1,newyorktelegraph.com,https://www.newyorktelegraph.com/news/27199509...,None,"{location_type=1, location_fullname=Chi

### Lets convert the indig column to integer and drop the gid column

In [77]:
def remove_nan_3(x):
    if str(x) == "nan":
        return 0
    else:
        return int(x)

FINAL_DATASET['indig'] = FINAL_DATASET['indig'].apply(lambda x: remove_nan_3(x))
FINAL_DATASET.drop(columns = ['gid'], inplace = True)
FINAL_DATASET.head(20)

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,tone,dates,amounts,translationinfo,country,year,lat_longs,Number of unique type 3 and 4 locations,GRID IDs,indig
0,20211202213000-16,2021-12-02 21:30:00,1,wmal.com,https://www.wmal.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,"[0, 0]",0,0,0
1,20211202213000-102,2021-12-02 21:30:00,1,redstate.com,https://redstate.com/nick-arama/2021/12/02/san...,None,"{location_type=4, location_fullname=Santiago, ...","[{organization=Sheriff Department, character_o...","[{theme=KILL, character_offset=142}, {theme=TA...","[{person=Hannah Reed, character_offset=1871}, ...","{tone=-3.85395537525355, positive_score=0.6085...",None,None,None,CI,2021,"[-33.25, -70.75]",1,81579,0
2,20211202213000-508,2021-12-02 21:30:00,1,kfru.com,https://www.kfru.com/news/repealing-roe-v-wade...,"[{count_type=PROTEST, count=3, object_type=, l...","{location_type=1, location_fullname=Chile, loc...","[{organization=Supreme Court, character_offset...","[{theme=TAX_FNCACT_WOMAN, character_offset=106...","[{person=Justin Trudeau, character_offset=2911...","{tone=-6.88888888888889, positive_score=1.9259...","[{date_resolution=1, month=0, day=0, year=1973...","[{amount=5, object=decades of constitutional p...",None,CI,2021,"[0, 0]",0,0,0
3,20211202213000-769,2021-12-02 21:30:00,1,thediplomat.com,https://thediplomat.com/2021/12/afghanistans-t...,"[{count_type=ARREST, count=2014, object_type=,...","{location_type=1, location_fullname=Bahamas, l...","[{organization=United Nations, character_offse...","[{theme=TAX_FNCACT_CANDIDATES, character_offse...","[{person=Ghulam Isaczai, character_offset=1780...","{tone=-1.9047619047619, positive_score=1.48148...","[{date_resolution=1, month=0, day=0, year=1949...","[{amount=2, object=situations, character_offse...",None,CI,2021,"[34.75, 69.25]",1,179779,0
4,20211202213000-843,2021-12-02 21:30:00,1,citywireamericas.com,https://citywireamericas.com/news/climate-chan...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=National Centers, character_off...","[{theme=NATURAL_DISASTER_EXTREME_WEATHER, char...","[{person=Michelle Dunstan, character_offset=21...","{tone=-1.57273918741809, positive_score=2.4246...",None,"[{amount=30, object=separate billion, characte...",None,CI,2021,"[-33.25, -70.75]",1,81579,0
5,20211202213000-1181,2021-12-02 21:30:00,1,realwire.com,https://www.realwire.com/releases/New-study-re...,None,"{location_type=1, location_fullname=Rwanda, lo...","[{organization=Portulans Institute, character_...",None,"[{person=Linkedin Youtube, character_offset=65...","{tone=0.931098696461825, positive_score=2.1415...","[{date_resolution=1, month=0, day=0, year=2002...","[{amount=130, object=global economies based on...",None,CI,2021,"[38.75, -77.25]",1,185246,0
6,20211202213000-1641,2021-12-02 21:30:00,1,theconversation.com,https://theconversation.com/a-new-exoplanet-me...,None,"{location_type=1, location_fullname=Chile, loc...",[{organization=Nasa Transiting Exoplanet Surve...,"[{theme=TAX_ETHNICITY_GERMAN, character_offset...",None,"{tone=0.757575757575757, positive_score=3.0303...",None,"[{amount=1000000, object=of stars being monito...",None,CI,2021,"[0, 0]",0,0,0
7,20211202213000-1764,2021-12-02 21:30:00,1,wcvb.com,https://www.wcvb.com/article/taco-night-space-...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=Kennedy Space Center, character...","[{theme=TAX_FNCACT_MANAGER, character_offset=5...","[{person=Matthias Maurer, character_offset=606...",

### Lets now select only the rows which have indig = 1

In [78]:
FINAL_DATASET = FINAL_DATASET[FINAL_DATASET['indig'] == 1].reset_index(drop=True)
FINAL_DATASET.head(20)

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,tone,dates,amounts,translationinfo,country,year,lat_longs,Number of unique type 3 and 4 locations,GRID IDs,indig
0,20211222200000-955,2021-12-22 20:00:00,1,wtbx.com,https://wtbx.com/2021/12/22/arson-attack-burns...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=Chile Congress, character_offse...","[{theme=GENERAL_GOVERNMENT, character_offset=1...","[{person=Sonya Hepinstall, character_offset=13...","{tone=-8.09523809523809, positive_score=0.4761...",None,"[{amount=30, object=properties were burned on,...",None,CI,2021,"[-38.75, -73.25]",2,73654,1
1,20211112064500-178,2021-11-12 06:45:00,1,nashvilleherald.com,https://www.nashvilleherald.com/news/271709970...,None,"{location_type=4, location_fullname=Nanaimo, B...","[{organization=Nevada Energy Metals, character...","[{theme=NATURAL_DISASTER_VOLCANIC, character_o...","[{person=Mount Sidney Williams, character_offs...","{tone=-0.384911470361817, positive_score=1.154...","[{date_resolution=1, month=0, day=0, year=1942...","[{amount=99, object=km north, character_offset...",None,CI,2021,"[49.25, -123.75]",12,200273,1
2,20211112064500-178,2021-11-12 06:45:00,1,nashvilleherald.com,https://www.nashvilleherald.com/news/271709970...,None,"{location_type=4, location_fullname=Nanaimo, B...","[{organization=Nevada Energy Metals, character...","[{theme=NATURAL_DISASTER_VOLCANIC, character_o...","[{person=Mount Sidney Williams, character_offs...","{tone=-0.384911470361817, positive_score=1.154...","[{date_resolution=1, month=0, day=0, year=1942...","[{amount=99, object=km north, character_offset...",None,CI,2021,"[40.75, -119.75]",12,188041,1
3,20211112064500-178,2021-11-12 06:45:00,1,nashvilleherald.com,https://www.nashvilleherald.com/news/271709970...,None,"{location_type=4, location_fullname=Nanaimo, B...","[{organization=Nevada Energy Metals, character...","[{theme=NATURAL_DISASTER_VOLCANIC, character_o...","[{person=Mount Sidney Williams, character_offs...","{tone=-0.384911470361817, positive_score=1.154...","[{date_resolution=1, month=0, day=0, year=1942...","[{amount=99, object=km north, character_offset...",None,CI,2021,"[40.25, -119.25]",12,187322,1
4,20210121163000-137,2021-01-21 16:30:00,1,scienceblog.com,https://horizon.scienceblog.com/1589/unravelli...,None,"{location_type=4, location_fullname=Lastarria,...","[{organization=University Of Potsdam, characte...","[{theme=NATURAL_DISASTER_VOLCANIC, character_o...","[{person=Luca Caricchi, character_offset=4983}...","{tone=-2.26817878585724, positive_score=1.2008...",None,"[{amount=60, object=kilometres, character_offs...",None,CI,2021,"[19.25, -99.25]",5,157122,1
5,20210309164500-145,2021-03-09 16:45:00,1,ctpost.com,https://www.ctpost.com/entertainment/article/Q...,"[{count_type=KILL, count=29, object_type=, loc...","{location_type=4, location_fullname=Isabel All...","[{organization=Associated Press, character_off...","[{theme=TAX_FNCACT_PATRIARCH, character_offset...","[{person=Rodrigo Bazaes, character_offset=643}...","{tone=-0.64935064935065, positive_score=2.7417...","[{date_resolution=1, month=0, day=0, year=1994...","[{amount=29, object=while, character_offset=61...",None,CI,2021,"[17.75, -97.25]",2,154966,1
6,20210309164500-145,2021-03-09 16:45:00,1,ctpost.com,https://www.ctpost.com/entertainment/article/Q...,"[{count_type=KILL, count=29, object_type=, loc...","{location_type=4, location_fullname=Isabel All...","[{organization=Associated Press, character_off...","[{theme=TAX_FNCACT_PATRIARCH, character_offset...","[{person=Rodrigo Bazaes, character_offset=643}...","{tone=-0.64935064935065, positive_score=2.7417...","[{date_resolution=1, month=0, day=0, year=1994...","[{amount=29, object=while, character_offset=61...",None,CI,2021,"[19.25, -99.25]",2,157122,1
7,20210309164500-174,2021-03-09 16:45:00,1,ctpost.com,https://www.ctpost.com/news/article/Isabel-All...,"[{count_type=K

## Now lets play around with the 'tone' column

In [79]:
FINAL_DATASET.shape

(93442, 20)

In [80]:
tone = FINAL_DATASET['tone']
tone

0        {tone=-8.09523809523809, positive_score=0.4761...
1        {tone=-0.384911470361817, positive_score=1.154...
2        {tone=-0.384911470361817, positive_score=1.154...
3        {tone=-0.384911470361817, positive_score=1.154...
4        {tone=-2.26817878585724, positive_score=1.2008...
                               ...                        
93437    {tone=-2.54237288135593, positive_score=3.1073...
93438    {tone=-5.14820592823713, positive_score=0.7800...
93439    {tone=0.341102899374645, positive_score=2.8993...
93440    {tone=0.341102899374645, positive_score=2.8993...
93441    {tone=-2.36813778256189, positive_score=2.3681...
Name: tone, Length: 93442, dtype: object

### Now lets extract the numbers from each of these rows or tone

In [81]:
import re

def tone_extractor(x):
    lst = re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x)
    return lst

tonal_data = pd.DataFrame(tone.apply(lambda x: tone_extractor(x)).to_list(), columns = ['tone', 'positive_score', 'negative_score', 'polarity', 'activity_reference_density', 'self_group_reference_density', 'word_count'])
tonal_data

,tone,positive_score,negative_score,polarity,activity_reference_density,self_group_reference_density,word_count
0,-8.09523809523809,0.476190476190476,8.57142857142857,9.04761904761905,13.3333333333333,0,199
1,-0.384911470361817,1.15473441108545,1.53964588144727,2.69438029253272,23.1716705157814,0.153964588144727,1235
2,-0.384911470361817,1.15473441108545,1.53964588144727,2.69438029253272,23.1716705157814,0.153964588144727,1235
3,-0.384911470361817,1.15473441108545,1.53964588144727,2.69438029253272,23.1716705157814,0.153964588144727,1235
4,-2.26817878585724,1.20080053368913,3.46897931954636,4.66977985323549,23.3488992661775,1.00066711140761,1381
...,...,...,...,...,...,...,...
93437,-2.54237288135593,3.10734463276836,5.64971751412429,8.75706214689266,19.2090395480226,0,620
93438,-5.14820592823713,0.78003120124805,5.92823712948518,6.70826833073323,21.216848673947,0.46801872074883,1186
93439,0.341102899374645,2.89937464468448,2.55827174530984,5.45764638999431,21.97271176805,1.05173393973849,3191
93440,0.341102899374645,2.89937464468448,2.55827174530984,5.45764638999431,21.97271176805,1.05173393973849,3191


In [83]:
FINAL_DATASET_READY = pd.concat([FINAL_DATASET, tonal_data], axis=1)
FINAL_DATASET_READY

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,...,Number of unique type 3 and 4 locations,GRID IDs,indig,tone,positive_score,negative_score,polarity,activity_reference_density,self_group_reference_density,word_count
0,20211222200000-955,2021-12-22 20:00:00,1,wtbx.com,https://wtbx.com/2021/12/22/arson-attack-burns...,None,"{location_type=1, location_fullname=Chile, loc...","[{organization=Chile Congress, character_offse...","[{theme=GENERAL_GOVERNMENT, character_offset=1...","[{person=Sonya Hepinstall, character_offset=13...",...,2,73654,1,-8.09523809523809,0.476190476190476,8.57142857142857,9.04761904761905,13.3333333333333,0,199
1,20211112064500-178,2021-11-12 06:45:00,1,nashvilleherald.com,https://www.nashvilleherald.com/news/271709970...,None,"{location_type=4, location_fullname=Nanaimo, B...","[{organization=Nevada Energy Metals, character...","[{theme=NATURAL_DISASTER_VOLCANIC, character_o...","[{person=Mount Sidney Williams, character_offs...",...,12,200273,1,-0.384911470361817,1.15473441108545,1.53964588144727,2.69438029253272,23.1716705157814,0.153964588144727,1235
2,20211112064500-178,2021-11-12 06:45:00,1,nashvilleherald.com,https://www.nashvilleherald.com/news/271709970...,None,"{location_type=4, location_fullname=Nanaimo, B...","[{organization=Nevada Energy Metals, character...","[{theme=NATURAL_DISASTER_VOLCANIC, character_o...","[{person=Mount Sidney Williams, character_offs...",...,12,188041,1,-0.384911470361817,1.15473441108545,1.53964588144727,2.69438029253272,23.1716705157814,0.153964588144727,1235
3,20211112064500-178,2021-11-12 06:45:00,1,nashvilleherald.com,https://www.nashvilleherald.com/news/271709970...,None,"{location_type=4, location_fullname=Nanaimo, B...","[{organization=Nevada Energy Metals, character...","[{theme=NATURAL_DISASTER_VOLCANIC, character_o...","[{person=Mount Sidney Williams, character_offs...",...,12,187322,1,-0.384911470361817,1.15473441108545,1.53964588144727,2.69438029253272,23.1716705157814,0.153964588144727,1235
4,20210121163000-137,2021-01-21 16:30:00,1,scienceblog.com,https://horizon.scienceblog.com/1589/unravelli...,None,"{location_type=4, location_fullname=Lastarria,...","[{organization=University Of Potsdam, characte...","[{theme=NATURAL_DISASTER_VOLCANIC, character_o...","[{person=Luca Caricchi, character_offset=4983}...",...,5,157122,1,-2.26817878585724,1.20080053368913,3.46897931954636,4.66977985323549,23.3488992661775,1.00066711140761,1381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93437,20210207064500-521,2021-02-07 06:45:00,1,thetelegraph.com,https://www.thetelegraph.com/news/article/Ecua...,"[{count_type=PROTEST, count=9, object_type=Yak...","{location_type=1, location_fullname=Venezuela,...","[{organization=Johns Hopkins University, chara...","[{theme=TAX_FNCACT_CANDIDATES, character_offse...","[{person=Guillermo Lasso, character_offset=138...",...,2,157122,1,-2.54237288135593,3.10734463276836,5.64971751412429,8.75706214689266,19.2090395480226,0,620
93438,20210904044500-87,2021-09-04 04:45:00,1,tribtown.com,http://www.tribtown.com/2021/09/03/ap-us-ecuad...,"[{count_type=KILL, count=19, object_type=, loc...","{location_type=1, location_fullname=Brazilians...","[{organization=White House, character_offset=3...","[{theme=TAX_FNCACT_GUIDE, character_offset=587...","[{person=Donald Trump, character_offset=3500},...",...,9,157122,1,-5.14820592823713,0.78003120124805,5.92823712948518,6.70826833073323,21.216848673947,0.46801872074883,1186
93439,20210502031500-255,2021-05-02 03:15:00,1,proactiveinvestors.com,http://www.proactiveinvestors.com/companies/ne...,None,"{location_type=2, location_fullname=Iowa, Unit...","[{organization=Glaxosmithkline, character_offs...","[{theme=CYBER_ATTACK, character_offset=7712}, ...","[{person=Michael Hewson, character_offset=5677...",...,10,100732,1,0.341102899374645,2.89937464468448,2.55827174530984,5.45764638999431,21

In [84]:
FINAL_DATASET_READY.to_csv('LATIN2.csv', index=False, header=True)